## Individual-based model (Polechova and Barton, 2015)

In [1]:
using Random, Distributions

In [2]:
#global variables
#r_m = maximum intrinsic growth rate
#V_s = variance of stabilising selection 

In [3]:
mutable struct agent
    #tag::Int64
    loci::Array
    #allele effect size ? -> L bi-allelic loci underlying trait x, allele effect size 0 or \alfa -> sum of allele effect sizes over all loci
end

In [4]:
mutable struct Agent{T}
    #tag::Int64
    loci::Vector{T}
    #allele effect size ? -> L bi-allelic loci underlying trait x, allele effect size 0 or \alfa -> sum of allele effect sizes over all loci
end

**Note 1**: just a style note, user defined types (structs) are usually capitalized (`CamelCase`) style.

**Note 2**: you have an `Array{Any}` field, which is inefficient, as the compiler cannot know which type is in the array in the `loci` field of an instance of `Agent`. You can specify the struct however very generally using a 'parametric type' `T` (or whatever symbol of course, typically uppercase. Also, in this case the loci are in a vector, while the `Array` type stands for a multidimensional array of any dimension (in julia lingo: `Array` == `Array{Any,N} where N`). It would be more efficient to use `Array{T,1}` or the synonymous `Vector{T}`. (Note, you could also store the loci in a ($n x 1$) matrix using `Array{T,2}` or `Matrix{T}`, which might be convenient if we go to higher ploidy levels later, but we can worry about that later)

Here is an illustration of what the difference is from the point of view of the compiler:

In [5]:
a1 = agent(randn(10))
a2 = Agent(randn(10));
typeof.([a1, a2])

2-element Array{DataType,1}:
 agent
 Agent{Float64}

Now we define the same function for both agent types:

In [6]:
pushlocus!(a::agent, x) = push!(a.loci, x)
pushlocus!(a::Agent, x) = push!(a.loci, x)

pushlocus! (generic function with 2 methods)

Now I use the `@code_warntype` macro to check how the compiler succeeds in inferring data types. It takes some time to get used to these outputs, but basically, things shown in red flag abstract types where the compiler failed at inferring the types to sufficient depth.

In [7]:
@code_warntype pushlocus!(a1, rand())

Variables
  #self#::Core.Compiler.Const(pushlocus!, false)
  a::agent
  x::Float64

Body::Array{T,1} where T
1 ─ %1 = Base.getproperty(a, :loci)::Array
│   %2 = Main.push!(%1, x)::Array{T,1} where T
└──      return %2


In [8]:
@code_warntype pushlocus!(a2, rand())

Variables
  #self#::Core.Compiler.Const(pushlocus!, false)
  a::Agent{Float64}
  x::Float64

Body::Array{Float64,1}
1 ─ %1 = Base.getproperty(a, :loci)::Array{Float64,1}
│   %2 = Main.push!(%1, x)::Array{Float64,1}
└──      return %2


In [9]:
#generate N haploid individuals with L loci and two possible allele-effect sizes α or 0
function make_agents(N,L,α)
#α = 1
#x = rand((0,1))
    #pop = [agent([α.*rand((0,1)) for i in 1:L]) for i in 1:N]
    pop = [Agent(α .* rand(Bool, L)) for i in 1:N]
    return pop
end

make_agents (generic function with 1 method)

Here you don't need the array comprehension, you can use `rand((0,1), L)` or `rand(Bool, L)` (note that in julia `true == 1` and `false == 0` and you can do e.g. `true * 0.1` )

In [10]:
agents = make_agents(10,10,1)

10-element Array{Agent{Int64},1}:
 Agent{Int64}([1, 1, 1, 1, 1, 0, 1, 0, 0, 1])
 Agent{Int64}([0, 1, 0, 1, 1, 0, 1, 0, 0, 0])
 Agent{Int64}([0, 0, 1, 1, 0, 0, 0, 0, 1, 0])
 Agent{Int64}([1, 0, 0, 1, 0, 0, 1, 0, 1, 1])
 Agent{Int64}([1, 0, 0, 1, 0, 1, 0, 1, 0, 0])
 Agent{Int64}([1, 0, 1, 0, 1, 0, 1, 0, 1, 1])
 Agent{Int64}([1, 1, 0, 1, 1, 1, 1, 1, 0, 1])
 Agent{Int64}([0, 0, 1, 0, 0, 0, 0, 0, 1, 0])
 Agent{Int64}([1, 1, 0, 1, 0, 1, 0, 0, 0, 0])
 Agent{Int64}([0, 0, 1, 0, 0, 1, 0, 0, 1, 0])

In [11]:
methods(agent)

# 2 methods for type constructor:
[1] agent(loci::Array) in Main at In[3]:3
[2] agent(loci) in Main at In[3]:3

In [12]:
methods(Agent)

# 1 method for type constructor:
[1] (::Type{Agent})(loci::Array{T,1}) where T in Main at In[4]:3

In [13]:
mutable struct deme
    agents::Array
    carrying_capacity::Int64
    optimal_phenotype::Int64 
end

In [14]:
struct Deme{A,T}
    agents::Vector{A}
    carrying_capacity::Int64
    optimal_phenotype::T 
end

Same comments as above, include parametric types where it makes sense. Also, `Int` type for optimal phenotype does not make much sense no? Also, I'm not sure whether it should be mutable, since you can push to arrays etc. in an immutable struct as well (you cannot change scalar fields though).

In [15]:
d = Deme(agents, 100, 0.0)

Deme{Agent{Int64},Float64}(Agent{Int64}[Agent{Int64}([1, 1, 1, 1, 1, 0, 1, 0, 0, 1]), Agent{Int64}([0, 1, 0, 1, 1, 0, 1, 0, 0, 0]), Agent{Int64}([0, 0, 1, 1, 0, 0, 0, 0, 1, 0]), Agent{Int64}([1, 0, 0, 1, 0, 0, 1, 0, 1, 1]), Agent{Int64}([1, 0, 0, 1, 0, 1, 0, 1, 0, 0]), Agent{Int64}([1, 0, 1, 0, 1, 0, 1, 0, 1, 1]), Agent{Int64}([1, 1, 0, 1, 1, 1, 1, 1, 0, 1]), Agent{Int64}([0, 0, 1, 0, 0, 0, 0, 0, 1, 0]), Agent{Int64}([1, 1, 0, 1, 0, 1, 0, 0, 0, 0]), Agent{Int64}([0, 0, 1, 0, 0, 1, 0, 0, 1, 0])], 100, 0.0)

we can make the display of the Deme nicer by defining a `show` method, here are some examples of typical 'overloading' of methods in Base. Note that these are 'one-liners' function definitions (i.e. these are function definition despite not having the `function` keyword)

In [16]:
Base.length(d::Deme) = length(d.agents)
Base.show(io::IO, d::Deme{A,T}) where {A,T} = write(io, "Deme{$A,$T}(N=$(length(d)))")
Base.push!(d::Deme, a::Agent) = push!(d.agents, a)

In [17]:
push!(d, Agent(1 .* rand(Bool, 10)))
d

Deme{Agent{Int64},Float64}(N=11)

In [18]:
mutable struct habitat
    demes::Array
end

In [19]:
struct Habitat{T}
    demes::Vector{T}
end

Same thing

In [20]:
#generate a single deme with population of size N, L loci, α allelic effect, carrying capacity K and optimal phenotype θ
function make_deme(N,L,α,K,θ)
    pop = make_agents(N,L,α)
    return Deme(pop,K,θ)
end

make_deme (generic function with 1 method)

In [21]:
w = make_deme(5,2,1,150,0.)

Deme{Agent{Int64},Float64}(N=5)

In [22]:
function pop_size(d::Deme)
    return length(d.agents)
end

pop_size (generic function with 1 method)

Note that this is equivalent to my overloading of `Base.length`.

In [23]:
pop_size(w)

5

In [24]:
#generate a linear 1D habitat of x demes with equal pop size (in paper the pop starts in center of habitat)
function make_habitat(x,N,L,α,K,θ)
    hab = [make_deme(N,L,α,K,θ) for i in 1:x]
    return Habitat(hab)
end

make_habitat (generic function with 1 method)

In [25]:
h = make_habitat(5,5,2,1,150,10)

Habitat{Deme{Agent{Int64},Int64}}(Deme{Agent{Int64},Int64}[Deme{Agent{Int64},Int64}(N=5), Deme{Agent{Int64},Int64}(N=5), Deme{Agent{Int64},Int64}(N=5), Deme{Agent{Int64},Int64}(N=5), Deme{Agent{Int64},Int64}(N=5)])

A nicer show method would be nice here as well:

In [26]:
Base.length(h::Habitat) = length(h.demes)
Base.show(io::IO, h::Habitat) = write(io, 
    "Habitat (n=$(length(h))):\n  $(join(string.(h.demes), "\n  "))")

In [27]:
h

Habitat (n=5):
  Deme{Agent{Int64},Int64}(N=5)
  Deme{Agent{Int64},Int64}(N=5)
  Deme{Agent{Int64},Int64}(N=5)
  Deme{Agent{Int64},Int64}(N=5)
  Deme{Agent{Int64},Int64}(N=5)

In [28]:
#habitat with 7 demes and starting pop in central deme
h = make_habitat(7,0,0,0,150,10)
p = make_agents(3,3,1)
# h.demes[4].agents = p  
# Note the above doesn't work for the immutable struct, but the following does
push!(h.demes[4].agents, agents...)
h

Habitat (n=7):
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=11)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)

Note the changes above to work with the immutable type, which is a somwehat more natural way to do the same. Also note the usage of the 'splat' operator `...` (look it up in the docs)

In [29]:
#should be diffusive migration with a Gaussian dispersal kernel (truncated at 2 SD)
function migrate(h::Habitat)
    new_h = make_habitat(length(h.demes),0,0,0,150,10)
    #step = rand((-1,0,1))
    i = 0
    for deme in h.demes
        i += 1
        for agent in deme.agents
            step = rand((-1,0,1))
            if step == -1
                push!(new_h.demes[i-1].agents, agent)
            elseif step == 0
                push!(new_h.demes[i].agents, agent)
            elseif step == 1
                push!(new_h.demes[i+1].agents, agent)
            end
        end
    end
    return new_h
end

migrate (generic function with 1 method)

Note that we defined the `push!` method for a `Deme`, so we can do those pushes somewhat more compactly (in julia, I'd say the less dots `.` you use, the better!).

Note that your method above goes out of bounds at the habitat edges!

Note that you can use enumerate instead of manually doing the increments of `i`.

Here's my version of your random walk migration function:

In [30]:
emptycopy(d::Deme{A}) where A = Deme(A[], d.carrying_capacity, d.optimal_phenotype)
emptycopy(h::Habitat) = Habitat(emptycopy.(h.demes))

function migrate(h::Habitat, p)
    new_h = emptycopy(h)
    for (i, deme) in enumerate(h.demes)
        for agent in deme.agents
            step = rand() < p ? rand([-1,1]) : 0 
            if step == -1 && i == 1
                step = 0
            elseif step == 1  && i == length(h)
                step = 0
            end
            push!(new_h.demes[i+step], agent)
        end
    end
    new_h
end

migrate (generic function with 2 methods)

This is a slightly generalized version, assuming random walk migration with probability $p/2$ of migrating to the right and $p/2$ of migrating to the left. It assumes absorbing boundaries, i.e. when at the edge of the habitat, the migration rate is $p/2$ instead of $p$, and all attempts to go over the edge are rejected and the agent stays in the deme where it is.

In [31]:
for i=1:5
    h = migrate(h, 0.2)
    @show h
end

h = Habitat (n=7):
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=3)
  Deme{Agent{Int64},Int64}(N=7)
  Deme{Agent{Int64},Int64}(N=1)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
h = Habitat (n=7):
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=2)
  Deme{Agent{Int64},Int64}(N=6)
  Deme{Agent{Int64},Int64}(N=3)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
h = Habitat (n=7):
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=3)
  Deme{Agent{Int64},Int64}(N=5)
  Deme{Agent{Int64},Int64}(N=3)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
h = Habitat (n=7):
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=1)
  Deme{Agent{Int64},Int64}(N=2)
  Deme{Agent{Int64},Int64}(N=4)
  Deme{Agent{Int64},Int64}(N=4)
  Deme{Agent{Int64},Int64}(N=0)
  Deme{Agent{Int64},Int64}(N=0)
h = Habitat (n=7):
  Deme{Ag